Парсер сайта "Реформа ЖКХ" https://www.reformagkh.ru/

Жилые дома Москвы. Интересует прежде всего, количество зарегистрированных жителей. Этих данных в ОД сайта нет.

Часть 2. Собирает информацию со страниц домов.

Сайт не очень охотно отдает данные, выдает капчу через каждые 300-800 запросов. Если нужен большой массив, лучше собирать данные через официальное api.

Последнее обновление парсера: 2021-08-02

### Собираю ссылки на муниципальные районы ###

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument('--proxy-server=??.??.???.???:?????') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)

In [ ]:
# загружаю датасет с ссылками

work_links = pd.read_excel('C:/Data/Реформа ЖКХ/buildings_links.xlsx', sheet_name='data')

In [ ]:
work_links.sample()

In [ ]:
# список для данных
data = []

In [ ]:
for i in tqdm(range(work_links.shape[0])):
    
    url = work_links.iloc[i][6] # беру ссылку из датафрейма links
    
    driver.get(url)
    sleep(3) # пауза, чтобы страница прогрузилась
    
    driver.implicitly_wait(5)
    address = work_links.iloc[i][2] # добавляю адрес из фрейма, чтобы не парсить второй раз
    
    house_info = driver.find_elements_by_class_name('house-specs') # забираю блок с данными
    
    info = []
    for j in house_info: # разбираю его циклом
        info.append([j.text])
        
    driver.find_element_by_tag_name('a[data-target="#address-changes-popup"]').click() # кликаю на историю адреса
    
    # сайт "тормозит" и не всегда успевает отдать данные, поэтому поставил явное ожидание
    element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, 'changes'))
    )
    
    # данные доступны, граблю
    address_history = driver.find_element_by_class_name('changes').text        
        
    data.append([address, info, address_history])

In [ ]:
# создаю датафрейм на основе списка
head_list = ['address', 'house_info', 'address_history']
work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.sample(3)

In [ ]:
work.shape

In [ ]:
work.to_excel('C:/Data/Реформа ЖКХ/mos_buildings_2.xlsx', encoding='utf-8', index=False)

In [ ]:
driver.close()